# Introduction 

In the following notebook, I will be exploring a cleaned version of the energy_dataset.csv file.

* Raw data can be found [here](https://github.com/KishenSharma6/Weather-Energy-Consumption-in-Spain/tree/master/Data/01_Raw_Data)
* Cleaned data can be found [here](https://github.com/KishenSharma6/Weather-Energy-Consumption-in-Spain/tree/master/Data/02_Cleaned_Data)

**Read in libraries for notebook**

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**Set notebook preferences**

In [5]:
#Set preferences for pandas 
pd.set_option("display.max_columns", 101)

#Set style for visualizations
plt.style.use('Solarize_Light2')

**Read in data**

In [8]:
#Set path to raw data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Spain Hourly Energy Demand and Weather'

#Read in raw data
df = pd.read_csv(path + '/Data/02_Cleaned_Data/2020_0505_Cleaned_Energy_Dataset.csv', index_col='time')

# Data Overview

* time: Datetime index localized to CET
* generation biomass: biomass generation in MW
* generation fossil brown coal/lignite: coal/lignite generation in MW
* generation fossil coal-derived gas: coal gas generation in MW
* generation fossil gas: gas generation in MW
* generation fossil hard coal: coal generation in MW
* generation fossil oil: oil generation in MW
* generation fossil oil shale: shale oil generation in MW
* generation fossil peat: peat generation in MW
* generation geothermal: geothermal generation in MW
* generation hydro pumped storage aggregated: hydro1 generation in MW
* generation hydro pumped storage consumption: hydro2 generation in MW
* generation hydro run-of-river and poundage: hydro3 generation in MW
* generation hydro water reservoir: hydro4 generation in MW
* generation marine: sea generation in MW
* generation nuclear: nuclear generation in MW
* generation other: other generation in MW
* generation other renewable: other renewable generation in MW
* generation solar: solar generation in MW
* generation waste: waste generation in MW
* generation wind offshore: wind offshore generation in MW
* generation wind onshore: wind onshore generation in MW
* forecast solar day ahead: forecasted solar generation
* forecast wind offshore eday ahead: forecasted offshore wind generation
* forecast wind onshore day ahead: forecasted onshore wind generation
* total load forecast: forecasted electrical demand
* total load actual: actual electrical demand
* price day ahead: forecasted price EUR/MWh
* price actual: price in EUR/MWh

**Data Preview**

In [9]:
#Print df shape
print('Shape of data:', df.shape)

#View head
df.head()

Shape of data: (35064, 20)


,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 01:00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 02:00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 03:00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 04:00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
